# Section 0: Import Dependencies

In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import time
from sqlalchemy import create_engine, func
from sqlalchemy.ext.automap import automap_base
import pymongo
from sqlalchemy.orm import Session
%matplotlib inline

# Import API key
from config import gkey

# Section 1: Read Datasets

In [3]:
banzuke = pd.read_csv('banzuke.csv')
banzuke.head()

,basho,id,rank,rikishi,heya,shusshin,birth_date,height,weight,prev,prev_w,prev_l
0,1983.01,1354,Y1e,Chiyonofuji,Kokonoe,Hokkaido,1955-06-01,182.0,116.0,Y1e,14.0,1.0
1,1983.01,4080,Y1w,Kitanoumi,Mihogaseki,Hokkaido,1953-05-16,179.0,165.0,Y2eHD,9.0,3.0
2,1983.01,4095,Y2eHD,Wakanohana,Futagoyama,Aomori,1953-04-03,186.0,133.0,Y1w,0.0,0.0
3,1983.01,4104,O1e,Takanosato,Futagoyama,Aomori,1952-09-29,181.0,144.0,O1e,10.0,5.0
4,1983.01,4112,O1w,Kotokaze,Sadogatake,Mie,1957-04-26,183.0,163.0,O1w,10.0,5.0


In [4]:
results = pd.read_csv('results.csv')
results.head()

,basho,day,rikishi1_id,rikishi1_rank,rikishi1_shikona,rikishi1_result,rikishi1_win,kimarite,rikishi2_id,rikishi2_rank,rikishi2_shikona,rikishi2_result,rikishi2_win
0,1983.01,1,4140,J13w,Chikubayama,0-1 (7-8),0,yorikiri,4306,Ms1e,Ofuji,1-0 (6-1),1
1,1983.01,1,4306,Ms1e,Ofuji,1-0 (6-1),1,yorikiri,4140,J13w,Chikubayama,0-1 (7-8),0
2,1983.01,1,1337,J12w,Tochitsukasa,1-0 (9-6),1,oshidashi,4323,J13e,Shiraiwa,0-1 (3-12),0
3,1983.01,1,4323,J13e,Shiraiwa,0-1 (3-12),0,oshidashi,1337,J12w,Tochitsukasa,1-0 (9-6),1
4,1983.01,1,4097,J12e,Tamakiyama,0-1 (8-7),0,yorikiri,4319,J11w,Harunafuji,1-0 (5-10),1


In [5]:
banzuke[banzuke['basho']==2021.03]['rikishi'].value_counts()

Mugendai      1
Sakurai       1
Harimanada    1
Oginohama     1
Kaizen        1
             ..
Daishoko      1
Aonoumi       1
Tanahashi     1
Nogami        1
Kakutaiki     1
Name: rikishi, Length: 654, dtype: int64

In [6]:
banzuke[(banzuke['basho']==2021.03) & (banzuke['rikishi']=='Terunofuji')]

,basho,id,rank,rikishi,heya,shusshin,birth_date,height,weight,prev,prev_w,prev_l
167785,2021.03,11927,S1e,Terunofuji,Isegahama,Mongolia,1991-11-29,192.0,158.5,S1e,11.0,4.0


In [7]:
results[(results['basho']==2021.03) & (results['rikishi1_shikona']=='Terunofuji')]

,basho,day,rikishi1_id,rikishi1_rank,rikishi1_shikona,rikishi1_result,rikishi1_win,kimarite,rikishi2_id,rikishi2_rank,rikishi2_shikona,rikishi2_result,rikishi2_win
218856,2021.03,1,11927,S1e,Terunofuji,1-0 (12-3),1,shitatenage,12239,M2e,Hokutofuji,0-1 (9-6),0
218896,2021.03,2,11927,S1e,Terunofuji,2-0 (12-3),1,oshidashi,12370,M2w,Wakatakakage,0-2 (10-5),0
218940,2021.03,3,11927,S1e,Terunofuji,3-0 (12-3),1,yoritaoshi,11985,K2w,Daieisho,0-3 (8-7),0
218980,2021.03,4,11927,S1e,Terunofuji,4-0 (12-3),1,kotenage,11946,M3e,Meisei,2-2 (10-5),0
219022,2021.03,5,11927,S1e,Terunofuji,4-1 (12-3),0,oshidashi,12043,M1w,Onosho,2-3 (4-11),1
219060,2021.03,6,11927,S1e,Terunofuji,5-1 (12-3),1,tsuridashi,12231,M4e,Kiribayama,2-4 (7-8),0
219102,2021.03,7,11927,S1e,Terunofuji,6-1 (12-3),1,yorikiri,12210,K1w,Mitakeumi,3-4 (8-7),0
219141,2021.03,8,11927,S1e,Terunofuji,6-2 (12-3),0,yorikiri,6480,K1e,Takayasu,7-1 (10-5),1
219183,2021.03,9,11927,S1e,Terunofuji,7-2 (12-3),1,kimedashi,11784,M4w,Myogiryu,5-4 (7-8),0
219222,2021.03,10,11927,S1e,Terunofuji,7-3 (12-3),0,tsukiotoshi,12024,M3w,Shimanoumi,3-7 (4-11),1


In [8]:
banzuke[(banzuke['rank'].str.startswith('O')) & (banzuke['basho']==2021.03)]

,basho,id,rank,rikishi,heya,shusshin,birth_date,height,weight,prev,prev_w,prev_l
167782,2021.03,12130,O1e,Shodai,Tokitsukaze,Kumamoto,1991-11-05,182.0,150.0,O1w,11.0,4.0
167783,2021.03,12291,O1w,Asanoyama,Takasago,Toyama,1994-03-01,189.0,158.0,O2e,11.0,4.0
167784,2021.03,12191,O2e,Takakeisho,Tokiwayama,Hyogo,1996-08-05,173.0,149.0,O1e,2.0,8.0


In [9]:
banzuke[(banzuke['rank'].str.contains('S\d')) & (banzuke['basho']==2021.03)]

,basho,id,rank,rikishi,heya,shusshin,birth_date,height,weight,prev,prev_w,prev_l
167785,2021.03,11927,S1e,Terunofuji,Isegahama,Mongolia,1991-11-29,192.0,158.5,S1e,11.0,4.0
167786,2021.03,11855,S1w,Takanosho,Tokiwayama,Chiba,1994-11-14,181.5,128.5,S1w,9.0,6.0


In [10]:
banzuke[(banzuke['rank'].str.contains('K\d')) & (banzuke['basho']==2021.03)]

,basho,id,rank,rikishi,heya,shusshin,birth_date,height,weight,prev,prev_w,prev_l
167787,2021.03,6480,K1e,Takayasu,Tagonoura,Ibaraki,1990-02-28,187.0,163.0,K1e,9.0,6.0
167788,2021.03,12210,K1w,Mitakeumi,Dewanoumi,Nagano,1992-12-25,179.0,149.0,K1w,9.0,6.0
167789,2021.03,11985,K2w,Daieisho,Oitekaze,Saitama,1993-11-10,179.0,135.7,M1w,13.0,2.0


In [11]:
banzuke[(banzuke['rank'].str.contains('M\d')) & (banzuke['basho']==2021.03)]

,basho,id,rank,rikishi,heya,shusshin,birth_date,height,weight,prev,prev_w,prev_l
167790,2021.03,11728,M1e,Takarafuji,Isegahama,Aomori,1987-02-18,186.0,161.1,M2e,9.0,6.0
167791,2021.03,12043,M1w,Onosho,Onomatsu,Aomori,1996-07-04,176.0,148.5,M3w,9.0,6.0
167792,2021.03,12239,M2e,Hokutofuji,Hakkaku,Saitama,1992-07-15,182.0,158.0,M1e,7.0,8.0
167793,2021.03,12370,M2w,Wakatakakage,Arashio,Fukushima,1994-12-06,180.0,115.0,M2w,0.0,0.0
167794,2021.03,11946,M3e,Meisei,Tatsunami,Kagoshima,1995-07-24,180.0,120.0,M7e,8.0,7.0
167795,2021.03,12024,M3w,Shimanoumi,Kise,Mie,1989-07-11,178.0,160.8,M10e,9.0,6.0
167796,2021.03,12231,M4e,Kiribayama,Michinoku,Mongolia,1996-04-24,184.0,94.0,M8w,8.0,7.0
167797,2021.03,11784,M4w,Myogiryu,Sakaigawa,Hyogo,1986-10-22,187.0,148.0,M9w,8.0,7.0
167798,2021.03,12055,M5e,Endo,Oitekaze,Ishikawa,1990-10-19,183.0,145.6,M5e,7.0,8.0
167799,2021.03,6463,M5w,Okinoumi,Hakkaku,Shimane,1985-07-29,190.0,154.0,M5w,7.0,8.0


In [12]:
banzuke[(banzuke['rank'].str.contains('Y\d')) & (banzuke['basho']==2021.03)]

,basho,id,rank,rikishi,heya,shusshin,birth_date,height,weight,prev,prev_w,prev_l
167780,2021.03,1123,Y1e,Hakuho,Miyagino,Mongolia,1985-03-11,193.0,150.7,Y1e,0.0,0.0
167781,2021.03,1219,Y1w,Kakuryu,Michinoku,Mongolia,1985-08-10,186.0,150.0,Y1w,0.0,0.0


In [13]:
topdiv = banzuke[(banzuke['rank'].str.contains('Y\d'))|(banzuke['rank'].str.contains('M\d'))|(banzuke['rank'].str.contains('K\d'))|(banzuke['rank'].str.contains('S\d'))|(banzuke['rank'].str.startswith('O'))]

In [14]:
topdiv.head()

,basho,id,rank,rikishi,heya,shusshin,birth_date,height,weight,prev,prev_w,prev_l
0,1983.01,1354,Y1e,Chiyonofuji,Kokonoe,Hokkaido,1955-06-01,182.0,116.0,Y1e,14.0,1.0
1,1983.01,4080,Y1w,Kitanoumi,Mihogaseki,Hokkaido,1953-05-16,179.0,165.0,Y2eHD,9.0,3.0
2,1983.01,4095,Y2eHD,Wakanohana,Futagoyama,Aomori,1953-04-03,186.0,133.0,Y1w,0.0,0.0
3,1983.01,4104,O1e,Takanosato,Futagoyama,Aomori,1952-09-29,181.0,144.0,O1e,10.0,5.0
4,1983.01,4112,O1w,Kotokaze,Sadogatake,Mie,1957-04-26,183.0,163.0,O1w,10.0,5.0


In [15]:
fighters = topdiv.copy()
fighters.drop_duplicates(subset='id',keep='last', inplace=True)

In [16]:
len(fighters)

331

In [17]:
fighters[fighters['rikishi']=='Terunofuji']

,basho,id,rank,rikishi,heya,shusshin,birth_date,height,weight,prev,prev_w,prev_l
167785,2021.03,11927,S1e,Terunofuji,Isegahama,Mongolia,1991-11-29,192.0,158.5,S1e,11.0,4.0


In [18]:
fighters.drop(['basho','prev','prev_w','prev_l'],axis=1,inplace=True)

In [19]:
fighters.head()

,id,rank,rikishi,heya,shusshin,birth_date,height,weight
2,4095,Y2eHD,Wakanohana,Futagoyama,Aomori,1953-04-03,186.0,133.0
36,4122,M14e,Kurosegawa,Isegahama,Tokyo,1951-05-13,184.0,130.0
37,4103,M14w,Banryuyama,Mihogaseki,Hyogo,1951-05-04,177.5,135.0
4492,4050,M8e,Takamiyama,Takasago,U.S.A.,1944-06-16,192.0,205.0
4497,4077,M10w,Fujizakura,Takasago,Yamanashi,1948-02-09,180.0,139.0


In [20]:
fighters.reset_index(drop=True, inplace=True)

In [21]:
fighters.head()

,id,rank,rikishi,heya,shusshin,birth_date,height,weight
0,4095,Y2eHD,Wakanohana,Futagoyama,Aomori,1953-04-03,186.0,133.0
1,4122,M14e,Kurosegawa,Isegahama,Tokyo,1951-05-13,184.0,130.0
2,4103,M14w,Banryuyama,Mihogaseki,Hyogo,1951-05-04,177.5,135.0
3,4050,M8e,Takamiyama,Takasago,U.S.A.,1944-06-16,192.0,205.0
4,4077,M10w,Fujizakura,Takasago,Yamanashi,1948-02-09,180.0,139.0


In [22]:
fighters.to_csv('fighters.csv',index=False)

# API Calls

In [23]:
stables = fighters.copy()

In [24]:
#Create needed columns to store data for each stable
stables['Lat'] =''
stables['Lng'] =''

#Perform API call to get information for each stable
base_url = 'https://maps.googleapis.com/maps/api/geocode/json?'


#Parameter for the API call, units will be handled in Metric system
params = {'key':gkey}

#Iterate through the list of stables to retrieve their information
for i, row in stables.iterrows():
    
    params['address'] = row['heya']+ ' Japan'

    response = requests.get(base_url,params=params).json()
    
    
    
    try:
        stables.loc[i,'Lat']= response['results'][0]['geometry']['location']['lat']
        stables.loc[i,'Lng']= response['results'][0]['geometry']['location']['lng']
    except:
        #Assign NaN to Cities not found to be dropped from the Dataframe
        stables.loc[i,['Lat','Lng']]= np.nan

KeyboardInterrupt: 

In [ ]:
response['results'][0]['geometry']['location']['lat']

In [ ]:
response['results'][0]['geometry']['location']['lng']

In [ ]:
stables.head(20)

In [ ]:
stables.reset_index(drop=True, inplace=True)

In [ ]:
stables.head()

In [ ]:
stables.to_csv('Stables.csv',index=False)

In [ ]:
stables.info()

In [ ]:
stables.head()

In [ ]:
results.head()

In [ ]:
results[(results['basho']==2021.03) & (results['rikishi2_shikona']=='Terunofuji')]

In [ ]:
stables.head()

In [ ]:
stables['Lat'] = stables['Lat'].astype('float64')

In [ ]:
stables['Lng'] = stables['Lng'].astype('float64')

In [ ]:
stables.info()

In [ ]:
stables.rename(columns={'rikishi':'fighter', 'heya':'dojo','shusshin':'district'}, inplace=True)

In [ ]:
stables.head()

In [ ]:
stables.to_csv('Stables.csv',index=False)

In [ ]:
results.info()

In [ ]:
results.tail()

In [ ]:
results['basho'] = results['basho'].astype('str')

In [ ]:
results.info()

In [ ]:
results.rename(columns={'basho':'tournament', 'rikishi1_id':'fighter1_id','rikishi1_rank':'fighter1_rank', \
                       'rikishi1_shikona':'fighter1_name', 'rikishi1_result':'fighter1_result', 'rikishi1_win':'fighter1_win', \
                       'kimarite':'finishing_move', 'rikishi2_id':'fighter2_id','rikishi2_rank':'fighter2_rank', \
                       'rikishi2_shikona':'fighter2_name', 'rikishi2_result':'fighter2_result', 'rikishi2_win':'fighter2_win'}, inplace=True)

In [ ]:
results.head()

In [ ]:
results.to_csv('Tournament_Results.csv',index=False)

In [29]:
results = pd.read_csv('Tournament_Results.csv')

In [30]:
results.head()

,tournament,day,fighter1_id,fighter1_rank,fighter1_name,fighter1_result,fighter1_win,finishing_move,fighter2_id,fighter2_rank,fighter2_name,fighter2_result,fighter2_win
0,1983.01,1,4140,J13w,Chikubayama,0-1 (7-8),0,yorikiri,4306,Ms1e,Ofuji,1-0 (6-1),1
1,1983.01,1,4306,Ms1e,Ofuji,1-0 (6-1),1,yorikiri,4140,J13w,Chikubayama,0-1 (7-8),0
2,1983.01,1,1337,J12w,Tochitsukasa,1-0 (9-6),1,oshidashi,4323,J13e,Shiraiwa,0-1 (3-12),0
3,1983.01,1,4323,J13e,Shiraiwa,0-1 (3-12),0,oshidashi,1337,J12w,Tochitsukasa,1-0 (9-6),1
4,1983.01,1,4097,J12e,Tamakiyama,0-1 (8-7),0,yorikiri,4319,J11w,Harunafuji,1-0 (5-10),1


In [31]:
stables = pd.read_csv('stables.csv')

In [32]:
stables.head()

,id,rank,fighter,dojo,district,birth_date,height,weight,Lat,Lng
0,4095,Y2eHD,Wakanohana,Futagoyama,Aomori,4/3/53,186.0,133.0,43.043111,141.316019
1,4122,M14e,Kurosegawa,Isegahama,Tokyo,5/13/51,184.0,130.0,32.413809,131.649506
2,4103,M14w,Banryuyama,Mihogaseki,Hyogo,5/4/51,177.5,135.0,35.495838,136.918181
3,4050,M8e,Takamiyama,Takasago,U.S.A.,6/16/44,192.0,205.0,34.766101,134.790601
4,4077,M10w,Fujizakura,Takasago,Yamanashi,2/9/48,180.0,139.0,34.766101,134.790601


In [44]:
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [65]:
#Connect to Postgress
rds_connection_string = "carlospazos@127.0.0.1:5432/sumo_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [62]:
engine.table_names()

['tournament_results', 'stables']

In [63]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

In [64]:
Base.classes.keys()

[]

In [38]:
stables.rename(columns={'Lat':'lat', 'Lng':'lng'}, inplace=True)

In [40]:
stables.head()

,id,rank,fighter,dojo,district,birth_date,height,weight,lat,lng
0,4095,Y2eHD,Wakanohana,Futagoyama,Aomori,4/3/53,186.0,133.0,43.043111,141.316019
1,4122,M14e,Kurosegawa,Isegahama,Tokyo,5/13/51,184.0,130.0,32.413809,131.649506
2,4103,M14w,Banryuyama,Mihogaseki,Hyogo,5/4/51,177.5,135.0,35.495838,136.918181
3,4050,M8e,Takamiyama,Takasago,U.S.A.,6/16/44,192.0,205.0,34.766101,134.790601
4,4077,M10w,Fujizakura,Takasago,Yamanashi,2/9/48,180.0,139.0,34.766101,134.790601


In [66]:
#Insert dataframes into Postgress DB
results.to_sql(name='tournament_results', con=engine, if_exists='append', index=False)
stables.to_sql(name='stables', con=engine, if_exists='append', index=False)